# PPD: Programação com CUDA

Hélio - DC/UFSCar - 2023

# Paralelização

Como visto, o modelo de paralelização em GPUs usando CUDA consiste em criar *kernels* (funções) cujo código é executado simultaneamente por múltiplas *threads*. Trata-se de um caso típico de decomposição de domnínio (ou de dados).

Na paralelização de um *loop for*, a estratégia é direta: ao invés de iterar N vezes, e se houver **1 *thead* para executar o código de cada iteração**?


```c
int N = 1<<10;

for (int i = 0; i < N; ++i) {
  a[i] = 2 * a[i];
}
```

Para isso, é preciso criar um *kernel* que execute o código de 1 iteração do *loop*. Basta agora ativar a execução deste *kernel* com um arranjo de *threads* que corresponda ao número de iterações.

É claro, contudo, que o código deste *kernel* deve ter algum mecanismo para que cada *thread* saiba qual é a iteração que deve executar; ou seja, quais elementos deve manipular.

```c
int N = 1<<10;

__global__ void loop() {
  // Em geral, a função do kernel corresponde ao trabalho de 1 iteração do loop.
  // Como determinar qual iteração?
  // Neste caso, há apenas 1 dimensão na organização das threads e o índice é direto

  int i = threadIdx.x;

  a[i] = 2 * a[i];
}

int main() {
  // ...

  loop <<<1, N>>>();

  // ...
  return 0;
}
```


Como há um limite para o número de *threads* em cada bloco (1024), para conseguir ter mais *threads* (1 para cada iteração), é preciso usar múltiplos blocos na ativação do *kernel*.

Uma estratégia é dividir o número de elementos (e iterações) pelo tamanho do bloco.

```c
int N = 1<<20;

for (int i = 0; i < N; ++i) {
  a[i] = 2 * a[i];
}
```

Considerando que cada bloco terá o número máximo de *threads* permitido, uma estratégia é fazer a divisão do número de iterações pelo tamanho dos blocos, o que vai resultar no número de blocos necessários.

O cálculo do índice que cada *thread* irá manipular, contudo, passa a ser calculado em função do número do bloco e desta *thread* dentro do bloco em que está.

```c
int N = 1<<20;

__global__ void loop() {
  // Em geral, a função do kernel corresponde ao trabalho de 1 iteração do loop.
  // Como determinar qual iteração?
  // Como há vários blocos, o índice pode ser obtido em função do
  // número do bloco vezes o tamanho de cada bloco, mais o índice da thread neste bloco
  
  int i = blockIdx.x * blockDim.x + threadIdx.x;


  a[i] = 2 * a[i];

  // Opa! É claro que 'a' deve ser uma variável em memória acessível pelo código na GPU...
}

int main() {
  // ...

  // falta a declaração de a e ajustes para que este vetor seja acessível
  // pelo código da GPU...
  ...

  int nthreads, nblocks;
  
  // supondo blocos com o tamanho máximo, organizados apenas na dimensão x
  nthreads = 1024;
  nblocks = (N + nthreads-1) / nthreads;

  loop <<< nblocks, nthreads >>>();

  // ... outras organizações de blocos poderiam ser usadas...

  // ...
  return 0;
}
```


No cálculo do número de blocos de *threads*, é possível que a divisão do número de elementos (iterações) pelo tamanho do bloco resulte num número de *threads* (blocos * tam blocos) maior que o número de iterações que precisam ser realizadas.

Assim, usando os mecanismos de cálculo do índice a manipular dentro da função do *kernel* executada pelas *threads*, pode ser que o índice resultante seja maior do que o total de elementos.

Deste modo, é preciso que no início de sua execução, cada *thread* determine o índice da iteração que corresponde aos seus identificadores (blockIdx.[x,y,z] e threadIdx.[x,y,z]) e verifique se este índice corresponde a uma iteração que precisa ser calculada. Caso contrário, esta *thread* não executa o código previsto, já que estaria manipulando elementos inexistentes ou iterações não previstas.


```c
int N = 1>>2000;

size_t threads_per_block = 128;

// ...

__global__ k_function() {

  int i = threadIdx.x + blockIdx.x * blockDim.x;
  if (i < N) { // Verifica se `i` corresponde a uma iteração válida (0..N-1)
    // executa código
  }
}

int main()
{
  // ...

  // É preciso haver ao menos N threads na grade, com apenas 1 bloco excedente
  size_t number_of_blocks = (N + threads_per_block - 1) / threads_per_block;

  k_function <<< number_of_blocks, threads_per_block>>> ();

  // ...
  return 0;
}
```

# Estudo de Caso: saxpy

O exemplo a seguir, extraído de https://developer.nvidia.com/blog/easy-introduction-cuda-c-and-c, ilustra uma implementação do programa SAXPY (Single-precision A * X Plus Y), onde se pode ver a manipulação de memória com alocação, cópias de e para a GPU e liberação do espaço.

In [ ]:
%%writefile saxpy.cu

#include <stdio.h>

__global__
void saxpy(int n, float a, float *x, float *y)
{
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  if (i < n)
    y[i] = a*x[i] + y[i];
}

int main(void)
{blockSize
  int N = 1<<20;
  float *x, *y, *d_x, *d_y;

  x = (float*)malloc(N*sizeof(float));
  y = (float*)malloc(N*sizeof(float));

  // alocação de memória na GPU
  cudaMalloc(&d_x, N*sizeof(float));
  cudaMalloc(&d_y, N*sizeof(float));

  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Cópia dos dados da memória RAM para a memória do dispositivo
  cudaMemcpy(d_x, x, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_y, y, N*sizeof(float), cudaMemcpyHostToDevice);

  // Perform SAXPY on 1M elements
  saxpy<<<(N+255)/256, 256>>>(N, 2.0f, d_x, d_y);

  // Cópia dos dados da memória da GPU para a memória RAM
  cudaMemcpy(y, d_y, N*sizeof(float), cudaMemcpyDeviceToHost);

  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = max(maxError, abs(y[i]-4.0f));
  printf("Max error: %f\n", maxError);

  // Liberação das áreas de memória alocadas da GPU
  cudaFree(d_x);
  cudaFree(d_y);
  free(x);
  free(y);
}

Writing saxpy.cu


# Calculando o tempo decorrido de processamento na GPU

Na programação com GPUs, uma estratégia para a paralelização é criar uma *thread* para executar cada uma das iterações dos laços contendo atividades independentes. A execução dessas *threads* de forma paralela, em rodadas executadas por até milhares de núcleos de processamento das GPUs, deve reduzir o tempo de execução.

Uma forma de medir o tempo total de execução da aplicação é usar o utilitário ***time*** na ativação do programa paralelo em linha de comando.

```
$ time ./prog-par

real	0m12.345s
user	0m0.369s
sys	 0m1.149s
```

Como há várias etapas e atividades para a execução de programas em GPU, contudo, pode ser interessante realizar medidas de tempo dentro do código, de forma a identificar tempos gastos com diferentes atividades, como transferências de memória e processamento, por exemplo.

Assim, uma outra opção para medir tempos associados às atividades é usar o mecanismo provido pelo SO para verificar o instante atual antes e após realizar as operações desejadas, como ilustrado a seguir:

```c
#include <sys/time.h>
...
struct timeval inic, fim;
// struct timespec inic, fim;
double etime;
...
// verifica instante atual antes do bloco de código a medir
gettimeofday(&inic, 0);    
// clock_gettime(CLOCK_REALTIME, &inic);

  // ativação do kernel
  kernel_call<<<dimGrid, dimBlock>>>();
  // mesmo retornando da função, não significa que o kernel já foi executado...

  // função que aguarda pela conclusão da última atividade submetida à execução na GPU
  cudaDeviceSynchronize();)
  // pronto, se retornou da chamada de sincronização, significa que a última operação na GPU (execução, neste caso) foi concluída

// verifica instante atual após a conclusão do bloco de código
gettimeofday(&fim, 0);
// clock_gettime(CLOCK_REALTIME, &fim);

// tempo decorrido: elapsed time
etime = (fim.tv_sec + fim.tv_usec/1e6) - (inic.tv_sec + inic.tv_usec/1e6),
// etime = (fim.tv_sec + fim.tv_nsec/1e9) - (inic.tv_sec + inic.tv_nsec/1e9);

```

Um aspecto a observar neste caso é que a execução da função de um *kernel* é assíncrona em relação ao processamento em CPU. Isso significa que para saber o instante em que a execução do *kernel* é concluída é preciso realizar uma operação de sincronização, como provido pela chamada [cudaDeviceSynchronize](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1g10e20b05a95f638a4071a655503df25d)().

Assim, esta técnica talvez não seja a mais adequada se quisermos medir tempos de atividades distintas na GPU sem realizar um bloqueio entre elas.

Há, contudo, outra forma de medir tempos gastos por atividades em GPU usando [eventos](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__EVENT.html).


<br>

## Medição de tempo usando eventos CUDA (cudaEvent)

Uma restrição à medição de tempo usando temporizadores e as sincronizações necessárias é que elas paralizam o *pipeline* da GPU. Como alternativa para as medições de tempo, CUDA oferece uma estrutura de [eventos](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__EVENT.html).  

Esta API inclui chamadas para criar eventos ([cudaEventCreate](http://docs.nvidia.com/cuda/cuda-runtime-api/index.html#group__CUDART__EVENT_1g320ab51604f3a7a082795202e7eaf774)), para registrar eventos ([cudaEventRecord](http://docs.nvidia.com/cuda/cuda-runtime-api/index.html#group__CUDART__EVENT_1ge31b1b1558db52579c9e23c5782af93e)) e para computar o tempo decorrido entre 2 eventos registrados, podendo ser usada como ilustrado a seguir.

```c
// declaração e iniciação das estruturas
cudaEvent_t start, stop;
cudaEventCreate(&start);
cudaEventCreate(&stop);
...
cudaMemcpy(d_x, x, N*sizeof(float), cudaMemcpyHostToDevice);
cudaMemcpy(d_y, y, N*sizeof(float), cudaMemcpyHostToDevice);

// antes de ativar a ação em GPU que se deseja medir, registra-se o evento
cudaEventRecord(start);

saxpy<<<(N+255)/256, 256>>>(N, 2.0f, d_x, d_y);

// registra-se o evento após a submissão de execução, neste caso.
cudaEventRecord(stop);
...
cudaMemcpy(y, d_y, N*sizeof(float), cudaMemcpyDeviceToHost);
...
// Espera-se pela conclusão do evento registrado após a execução do kernel
cudaEventSynchronize(stop);
float milliseconds = 0;
cudaEventElapsedTime(&milliseconds, start, stop);
```

A chamada [cudaEventRecord](http://docs.nvidia.com/cuda/cuda-runtime-api/index.html#group__CUDART__EVENT_1ge31b1b1558db52579c9e23c5782af93e) tem o papel de inserir o evento indicado no fluxo de ações padrão da GPU. Assim, a GPU irá registrar o instante associado a um evento **assim que processá-lo**.

Neste caso, o evento associado a ***start*** será processado antes de ativar a execução do kernel e o evento ***stop*** será processado depois que a execução do *kernel* for concluída, de modo que não é preciso que o código em CPU fique bloqueado à espera do evento. Assim, a função [cudaEventSynchronize](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__EVENT.html#group__CUDART__EVENT_1g949aa42b30ae9e622f6ba0787129ff22)() pode ser usada somente quando houver mais eventos a ativar no *kernel* e for relevante esperar pelo registro do tempo.

Já a função [cudaEventElapsedtime](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__EVENT.html#group__CUDART__EVENT_1g40159125411db92c835edb46a0989cd6)() retorna o número de milissegundos (com precisão de 0.5 microssegundos) decorridos entre os eventos indicados.














In [ ]:
%%writefile saxpy-t.cu

#include <stdio.h>

__global__
void saxpy(int n, float a, float *x, float *y)
{
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  if (i < n)
    y[i] = a*x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y, *d_x, *d_y;

	cudaEvent_t start, smanagedMemorytop;  // eventos para registro do tempo

	// cria eventos
	cudaEventCreate(&start);
	cudaEventCreate(&stop);

	// alocação dos vetores na memória do host (RAM)
  x = (float*)malloc(N*sizeof(float));
  y = (float*)malloc(N*sizeof(float));

  // alocação dos vetors na memória na GPU
  cudaMalloc(&d_x, N*sizeof(float));
  cudaMalloc(&d_y, N*sizeof(float));

	// iniciação dos elementos dos vetores na memória RAM
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Cópia dos dados da memória RAM para a memória do dispositivo
  cudaMemcpy(d_x, x, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_y, y, N*sizeof(float), cudaMemcpyHostToDevice);

	// registra evento antes da execução do kernel
	cudaEventRecord(start);

    // Perform SAXPY on 1M elements
    saxpy<<<(N+255)/256, 256>>>(N, 2.0f, d_x, d_y);

	// registra o evento após a execução do kernel
	cudaEventRecord(stop);

  // Cópia dos dados da memória da GPU para a memória RAM
  cudaMemcpy(y, d_y, N*sizeof(float), cudaMemcpyDeviceToHost);

  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = max(maxError, abs(y[i]-4.0f));
  printf("Max error: %f\n", maxError);

  // Liberação das áreas de memória alocadas da GPU
  cudaFree(d_x);
  cudaFree(d_y);
  free(x);
  free(y);

	// garante que o evento stop já ocorreu
	cudaEventSynchronize(stop);

	float milliseconds = 0;
	cudaEventElapsedTime(&milliseconds, start, stop);
	printf("Tempo de Execução na GPU: %.4f ms", milliseconds);

	return 0;
}

Writing saxpy-t.cu


In [ ]:
! nvcc saxpy-t.cu -o saxpy-t -O3
! time ./saxpy-t

Max error: 0.000000
Tempo de Execução na GPU: 0.0562 ms
real	0m0.938s
user	0m0.023s
sys	0m0.845s


In [ ]:
%%writefile saxpy-t2.cu

#include <stdio.h>

__global__
void saxpy(int n, float a, float *x, float *y)
{
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  if (i < n)
    y[i] = a*x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y, *d_x, *d_y;

	cudaEvent_t memcopy_i, memcopy_f, code_i, code_f;  // eventos para registro do tempo apresenta
	// cria eventos
	cudaEventCreate(&memcopy_i);
	cudaEventCreate(&memcopy_f);
	cudaEventCreate(&code_i);
	cudaEventCreate(&code_f);

	// alocação dos vetores na memória do host (RAM)
  x = (float*)malloc(N*sizeof(float));
  y = (float*)malloc(N*sizeof(float));

  // alocação dos vetors na memória na GPU
  cudaMalloc(&d_x, N*sizeof(float));
  cudaMalloc(&d_y, N*sizeof(float));

	// iniciação dos elementos dos vetores na memória RAM
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // registra evento antes da cópia de memória
  cudaEventRecord(memcopy_i);

  // Cópia dos dados da memória RAM para a memória do dispositivo
  cudaMemcpy(d_x, x, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_y, y, N*sizeof(float), cudaMemcpyHostToDevice);

  // registra evento após a cópia de memória
  cudaEventRecord(memcopy_f);

	// registra evento antes da execução do kernel
	cudaEventRecord(code_i);

    // Perform SAXPY on 1M elements
    saxpy<<<(N+255)/256, 256>>>(N, 2.0f, d_x, d_y);

	// registra o evento após a execução do kernel
	cudaEventRecord(code_f);

  // Cópia dos dados da memória da GPU para a memória RAM
  cudaMemcpy(y, d_y, N*sizeof(float), cudaMemcpyDeviceToHost);

  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = max(maxError, abs(y[i]-4.0f));
  printf("Max error: %f\n", maxError);

  // Liberação das áreas de memória alocadas da GPU
  cudaFree(d_x);
  cudaFree(d_y);
  free(x);
  free(y);

	// garante que o evento code_f já ocorreu
	cudaEventSynchronize(code_f);

	float milliseconds = 0;
	cudaEventElapsedTime(&milliseconds, memcopy_i, memcopy_f);
	printf("Tempo para transferência de memória host->device: %.4f ms\n", milliseconds);
	cudaEventElapsedTime(&milliseconds, code_i, code_f);
	printf("Tempo de Execução na GPU: %.4f ms\n", milliseconds);

	return 0;
}

Writing saxpy-t2.cu


In [ ]:
! if [ ! saxpy-t2 -nt saxpy-t2.cu ]; then nvcc saxpy-t2.cu -o saxpy-t2 -O3; fi
! ./saxpy-t2

Max error: 0.000000
Tempo para transferência de memória host->device: 2.9311 ms
Tempo de Execução na GPU: 128.3095 ms


## Medindo outros valores associados à execução do programa

***Obs***: Vou procurar criar uma seção específica sobre *profiling* mas, por ora, que tal examinar outros valores associados aos tempos de execução de um programa em GPU?

In [ ]:
! nvprof ./saxpy-t2

==1954== NVPROF is profiling process 1954, command: ./saxpy-t2
Max error: 0.000000
Tempo para transferência de memória host->device: 2.1941 ms
Tempo de Execução na GPU: 0.3047 ms
==1954== Profiling application: ./saxpy-t2
==1954== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   73.78%  1.6676ms         2  833.79us  813.91us  853.66us  [CUDA memcpy HtoD]
                   24.16%  546.17us         1  546.17us  546.17us  546.17us  [CUDA memcpy DtoH]
                    2.06%  46.495us         1  46.495us  46.495us  46.495us  saxpy(int, float, float*, float*)
      API calls:   98.17%  226.62ms         4  56.655ms     811ns  226.62ms  cudaEventCreate
                    1.27%  2.9407ms         3  980.24us  831.23us  1.1199ms  cudaMemcpy
                    0.16%  372.90us         2  186.45us  163.84us  209.06us  cudaFree
                    0.14%  330.68us         1  330.68us  330.68us  330.68us  cudaLaunchKernel
     

Observem a diferença de tempo entre as operações de cópia de memória e processamento!!!!

# Saxpy com cudaMallocManaged

Que tal experimentar o programa saxpy usando memória unificada?

O que muda? cudaMalloc -> cudaMallocManaged


In [ ]:
%%writefile saxpy.cu

#include <stdio.h>

__global__
void saxpy(int n, float a, float *x, float *y)
{
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  if (i < n)
    y[i] = a*x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y, *d_x, *d_y;

  x = (float*)malloc(N*sizeof(float));
  y = (float*)malloc(N*sizeof(float));

  // alocação de memória na GPU
  cudaMalloc(&d_x, N*sizeof(float));
  cudaMalloc(&d_y, N*sizeof(float));

  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Cópia dos dados da memória RAM para a memória do dispositivo
  cudaMemcpy(d_x, x, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_y, y, N*sizeof(float), cudaMemcpyHostToDevice);

  // Perform SAXPY on 1M elements3.0153ms
  saxpy<<<(N+255)/256, 256>>>(N, 2.0f, d_x, d_y);

  // Cópia dos dados da memória da GPU para a memória RAM
  cudaMemcpy(y, d_y, N*sizeof(float), cudaMemcpyDeviceToHost);

  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = max(maxError, abs(y[i]-4.0f));
  printf("Max error: %f\n", maxError);

  // Liberação das áreas de memória alocadas da GPU
  cudaFree(d_x);
  cudaFree(d_y);
  free(x);
  free(y);
}

Writing saxpy.cu


In [ ]:
! nvcc saxpy.cu -o saxpy -O3
! nvprof ./saxpy

==1209== NVPROF is profiling process 1209, command: ./saxpy
Max error: 0.000000
==1209== Profiling application: ./saxpy
==1209== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   68.28%  1.5931ms         2  796.57us  786.46us  806.68us  [CUDA memcpy HtoD]
                   29.72%  693.47us         1  693.47us  693.47us  693.47us  [CUDA memcpy DtoH]
                    2.00%  46.624us         1  46.624us  46.624us  46.624us  saxpy(int, float, float*, float*)
      API calls:   65.26%  243.79ms         2  121.89ms  122.91us  243.66ms  cudaMalloc
                   33.66%  125.73ms         1  125.73ms  125.73ms  125.73ms  cudaLaunchKernel
                    0.83%  3.1177ms         3  1.0392ms  984.12us  1.1224ms  cudaMemcpy
                    0.18%  686.42us         2  343.21us  257.13us  429.29us  cudaFree
                    0.06%  211.98us       114  1.8590us     179ns  77.730us  cuDeviceGetAttribute
              

Nos resultados acima, somar os tempos para as operações de cópia de memória e o tempo para a execução da função do *kernel*.

E.g.: 1.4568 + 0.4896 + 0.049151 = 1.995551 ms

In [ ]:
%%writefile saxpy-uni.cu

#include <stdio.h>

__global__
void saxpy(int n, float a, float *x, float *y)
{
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  if (i < n)
    y[i] = a*x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // alocação de memória na GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Perform SAXPY on 1M elements
  saxpy<<<(N+255)/256, 256>>>(N, 2.0f, x, y);

  cudaDeviceSynchronize();

  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = max(maxError, abs(y[i]-4.0f));
  printf("Max error: %f\n", maxError);

  // Liberação das áreas de memória alocadas da GPU
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Overwriting saxpy-uni.cu


In [ ]:
! nvcc saxpy-uni.cu -o saxpy-uni -O3
! nvprof ./saxpy-uni

==2034== NVPROF is profiling process 2034, command: ./saxpy-uni
Max error: 0.000000
==2034== Profiling application: ./saxpy-uni
==2034== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  2.8983ms         1  2.8983ms  2.8983ms  2.8983ms  saxpy(int, float, float*, float*)
      API calls:   98.23%  211.08ms         2  105.54ms  39.164us  211.04ms  cudaMallocManaged
                    1.36%  2.9218ms         1  2.9218ms  2.9218ms  2.9218ms  cudaDeviceSynchronize
                    0.23%  486.53us         2  243.26us  214.53us  272.00us  cudaFree
                    0.11%  229.43us         1  229.43us  229.43us  229.43us  cudaLaunchKernel
                    0.07%  140.10us       114  1.2280us     136ns  61.471us  cuDeviceGetAttribute
                    0.01%  11.880us         1  11.880us  11.880us  11.880us  cuDeviceGetName
                    0.00%  5.3500us         1  5.3500us  5.3500us  5.3500us  cuDeviceGe

No caso de uso de memória unificada, os tempos de cópia de memória ficam embutidos no tempo de execução do *kernel*.

Assim, no caso acima, T = 3.0153

Resumidamente, o tempo foi um pouco pior para memória unificada. Para casos em que há um maior uso de dados dentro da GPU, contudo, essa diferença tende a desaparecer.